In [1]:
import tensorflow as tf
import numpy as np
import cv2
import re

import src.model
import inspect_checkpoint
%load_ext autoreload
%autoreload 2

In [2]:
IMG_FILE_TYPE = "png"
MEGA_MODEL_WEIGHTS = './megadepth_model_fuse_bn_name/mega_prepost.ckpt'
tf.compat.v1.disable_eager_execution()

In [3]:
(train_images, train_labels), _ = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
with tf.device("CPU:0"):
    x = tf.zeros([4,384,512,3], tf.float32)
    C1 = depth_net.Hourglass()
    y = C1.call(x)
    print(C1.name)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
hourglass


In [5]:
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto())
init = tf.compat.v1.global_variables_initializer()
sess.run(init)
model_restore_var = [v for v in tf.compat.v1.global_variables()]

In [6]:
def load(saver, sess, ckpt_path):
    '''Load trained weights.
    
    Args:
      saver: TensorFlow saver object.
      sess: TensorFlow session.
      ckpt_path: path to checkpoint file with parameters.
    ''' 
    saver.restore(sess, ckpt_path)
    print("Restored model parameters from {}".format(ckpt_path))

In [7]:
# print(tf.compat.v1.global_variables())
trainedWeights = np.load('weights.pkl.npy',allow_pickle=True)[()]
# loader = tf.compat.v1.train.Saver(var_list=model_restore_var)
# load(loader, sess, MEGA_MODEL_WEIGHTS)


In [8]:
a = trainedWeights['3/0/0/3/0/0/3/0/1/3/0/1/3/conv2d_1/bias']
K = trainedWeights.keys()
print([k for k in K if '0/0/0' in k])

['3/0/0/3/0/0/3/0/0/0/batch_normalization/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_1/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_1/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_2/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_2/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_3/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_3/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_4/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_4/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_5/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_5/moving_variance', '3/0/0/3/0/0/3/0/0/0/batch_normalization_6/moving_mean', '3/0/0/3/0/0/3/0/0/0/batch_normalization_6/moving_variance', '3/0/0/3/0/0/3/0/0/0/conv2d/bias', '3/0/0/3/0/0/3/0/0/0/conv2d/kernel', '3/0/0/3/0/0/3/0/0/0/conv2d_1/bias', '3/0/0/3/0/0/3/0/0/0/conv2d_1/kernel', '3/0/0/3/0/0/3/0/0/0/conv2d_2

In [21]:
def getVariablePath(opPath):
    """
        returns the path of the operation without the
        inception, channel and module path polution
    """
    s1 = str(opPath).replace('module/', '')
    s2 = re.sub('channel\d+/', '', s1)
    # s3 = s2.replace('inception_','inception')
    s4 = re.sub('inception_*(base_)*\d*/*', '', s2)
    s5 = s4.replace(':0','')
    return s5

In [22]:
for var in tf.compat.v1.trainable_variables('module'):
    ckpt_path = getVariablePath(var.name)
    print(var.name,"\t\t\t\t", ckpt_path in K, "  ", ckpt_path)
    var.value = (trainedWeights[ckpt_path])


module/0/conv2d/kernel:0 				 True    0/conv2d/kernel
module/0/conv2d/bias:0 				 True    0/conv2d/bias
module/1/batch_normalization/gamma:0 				 True    1/batch_normalization/gamma
module/1/batch_normalization/beta:0 				 True    1/batch_normalization/beta
module/3/channel4/0/0/1/inception/conv2d/kernel:0 				 True    3/0/0/1/conv2d/kernel
module/3/channel4/0/0/1/inception/conv2d/bias:0 				 True    3/0/0/1/conv2d/bias
module/3/channel4/0/0/1/inception/inception__base/conv2d_1/kernel:0 				 False    3/0/0/1/base/conv2d_1/kernel


KeyError: '3/0/0/1/base/conv2d_1/kernel'

In [11]:

S = "module/3/channel4/0/0/1/inception/"

S5 = getVariablePath(S)
print(S5)
for k in K:
    if S5 in k:
        print(k)

3/0/0/1/
3/0/0/1/batch_normalization/moving_mean
3/0/0/1/batch_normalization/moving_variance
3/0/0/1/batch_normalization_1/moving_mean
3/0/0/1/batch_normalization_1/moving_variance
3/0/0/1/batch_normalization_2/moving_mean
3/0/0/1/batch_normalization_2/moving_variance
3/0/0/1/batch_normalization_3/moving_mean
3/0/0/1/batch_normalization_3/moving_variance
3/0/0/1/batch_normalization_4/moving_mean
3/0/0/1/batch_normalization_4/moving_variance
3/0/0/1/batch_normalization_5/moving_mean
3/0/0/1/batch_normalization_5/moving_variance
3/0/0/1/batch_normalization_6/moving_mean
3/0/0/1/batch_normalization_6/moving_variance
3/0/0/1/conv2d/bias
3/0/0/1/conv2d/kernel
3/0/0/1/conv2d_1/bias
3/0/0/1/conv2d_1/kernel
3/0/0/1/conv2d_2/bias
3/0/0/1/conv2d_2/kernel
3/0/0/1/conv2d_3/bias
3/0/0/1/conv2d_3/kernel
3/0/0/1/conv2d_4/bias
3/0/0/1/conv2d_4/kernel
3/0/0/1/conv2d_5/bias
3/0/0/1/conv2d_5/kernel
3/0/0/1/conv2d_6/bias
3/0/0/1/conv2d_6/kernel
3/0/0/3/0/0/1/batch_normalization/moving_mean
3/0/0/3/0/0/1/b